In [4]:
import collections
import re
from d2l import torch as d2l

In [6]:
#@save
# 把这个文字信息导入进来
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    # 一行一行读进来，然后不是英文字符的，全部变成空格
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [3]:
# 一行字符串直接扔进去肯定是没办法训练的，所以我们要把这一行行字符串转化成token，这个大名鼎鼎的东西
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        # 一行，拆成每个词，用空格拆分
        return [line.split() for line in lines]
    elif token == 'char':
        # 这玩意就是简单暴力的做成字符串......
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [4]:
# 这里的逻辑很简单，因为深度学习或者机器学习本质上还是一个数学问题，所以我们总是想要将问题用一个比较数学化的语句来表示
# 这里就是每个词，我生成一个数字去对应，形成一个映射关系
class Vocab:  #@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        # min_freq这玩意其实是，如果把word作为token，那有些词其实出现的次数很少，我们这时候就把这些词统一当作unknown
        # 减少要训练的量
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

上面我们使用了一个新的装饰器@property，下面是一些说明

在Python中，`@property`装饰器用于将一个方法转换为属性，这样你就可以像访问属性一样来访问这个方法，而不需要在后面加上括号`()`。

使用`@property`装饰器的好处是，你可以在访问属性的同时执行一些额外的代码，例如进行数据验证或计算。

以下是一个简单的示例：



In [5]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

# 注意到了吗，我们用self._radius这里其实定义了一个内部属性，这东西是不建议在外部访问的
# 但是我们下面通过def radius(self)这个函数让可以在外部用访问属性的方法来访问它
# 包括下面的def diameter(self),我们可以直接用self._radius来进行计算,但是不用直接访问内部的self._radius,从而一定程度上保证内部self._radius属性的安全
    @property
    def radius(self):
        return self._radius

    @property
    def diameter(self):
        return self._radius * 2

    @property
    def area(self):
        return 3.14 * self._radius ** 2



在这个例子中，`Circle`类有一个`_radius`属性和三个方法：`radius`，`diameter`和`area`。这三个方法都被`@property`装饰器修饰，所以你可以像访问属性一样来访问它们：



In [6]:
c = Circle(5)
print(c.radius)  # 输出: 5
print(c.diameter)  # 输出: 10
print(c.area)  # 输出: 78.5

5
10
78.5




你可以看到，`diameter`和`area`方法在被访问时会计算并返回圆的直径和面积，而不需要在后面加上括号。这使得代码更加简洁和直观。

In [7]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [8]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [9]:
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)